## Let's try Single Block Encoding Approach:

Why? If the order of ATCG doesn't matter then Single block encoding is proven to be the best compression method possible! just have a table with all possible 13-bit encoding (8192 types) which cost at most 15 bytes$*$8192 = 123 kb then compress 11671499 by 13 times to a total of only 1020687 = 1.02 MB (This comes from minimize $11671499/x+ (x+2)*(2^x)$, note that 12-bit is also good). The idea is to solve file_size$/x+ (x+2)*(2^x)$ But this is not realistic since we still need to represent these 8192 types with one symbol each.

For instance, we convert ATATGATCGA to 101001001, 110100,  101, 1 then to 2331, 123, 12, 1?

In [1]:
from sbe import SBE_f,SBE_b,SBE_convert_f,SBE_convert_b

temp = SBE_convert_f('ATCATASFEGATCG')
SBE_convert_b(temp)

temp = SBE_f([15,16,15,14,14,14,15,16,15,14,14,14])
print(temp)
SBE_b(temp)

[15, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 16, 1, 0, 0, 0, 1, 0, 0, 0, 14, 1, 1, 1, 1, 1, 1]


array([15., 16., 15., 14., 14., 14., 15., 16., 15., 14., 14., 14.])

In [2]:
import arithc as arith
import fqt, ppm
import contextlib, sys
import filecmp
from IPython.display import clear_output

In [39]:
def SBEcompress(inp, bitout):
    initfreqs = fqt.FlatFrequencyTable(257)
    model = fqt.SimpleFrequencyTable(initfreqs)
    enc = arith.ArithmeticCoder(32)
    enc.start_encode(bitout) # New line!
    s = []
    while True:
        symbol = inp.read(1)
        if len(symbol) == 0:
                break
        s += [symbol[0]+2] ## leave room for 0,1
    sbe = SBE_f(s)
    sbe_list = sbe
    print(len(sbe))
    idx = 0
#     # prior
#     model.set(0,1000000)
#     model.set(1,1000000)
    while True:
        # Read and encode one byte
        if idx == len(sbe):
            break
        symbol = sbe_list[idx]
        idx += 1
        if idx % 1100000 == 0:
            print(str(10*idx//(len(sbe)//10)) + ' percent done')
            clear_output(wait = True)
        t = model.get_total() ## New lines!
        l = model.get_low(symbol)
        h = model.get_high(symbol)
        enc.storeRegion(l,h,t) 
        model.increment(symbol)
    t = model.get_total() ## New lines!
    l = model.get_low(256)
    h = model.get_high(256)
    enc.storeRegion(l,h,t)
    enc.finish_encode(bitout)  # New line!
inputfile, outputfile = 'data\ecoli\Ecoli.txt', 'data\ecoli\Ecoli_sbe_compressed.txt'

#Perform file compression
with open(inputfile, "rb") as inp, \
        contextlib.closing(arith.BitOutputStream(open(outputfile, "wb"))) as bitout:
    SBEcompress(inp, bitout)

94 percent done


In [5]:
def decompress(bitin, out):
    initfreqs = fqt.FlatFrequencyTable(257)
    model = fqt.SimpleFrequencyTable(initfreqs)
    dec = arith.ArithmeticCoder(32)
    dec.start_decode(bitin) # New line!
    sbe = []
    while True:
        # Decode and write one byte
        total = model.get_total()
        Range = dec.R
        offset = dec.getTarget()#dec.D - dec.L
        #value_0 = ((offset + 1) * total - 1)// Range
        value = dec.getTarget(total)

        start = 0
        end = model.get_symbol_limit()
        while end - start > 1:
            middle = (start + end) >> 1
            if model.get_low(middle) > value:
                end = middle
            else:
                start = middle

        symbol = start

        #self 
        l = model.get_low(symbol) 
        h = model.get_high(symbol)
        dec.loadRegion(l,h,total)
        #print(symbol)
        model.increment(symbol)
        if symbol == 256:  # EOF symbol
            break
        sbe += [symbol]
    print(len(sbe))
    s_temp = SBE_b(sbe)
    s = [int(x - 2) for x in s_temp]
    print(len(s))
    for symbol in s:
        out.write(bytes((symbol,)))
    return sbe, s

        
inputfile, outputfile = 'data\ecoli\Fake_Ecoli_sbe_compressed.txt', 'data\ecoli\Fake_Ecoli_sbe_decompressed.txt'

# Perform file decompression
with open(inputfile, "rb") as inp, open(outputfile, "wb") as out:
    bitin = arith.BitInputStream(inp)
    sbe, s = decompress(bitin, out)


filecmp.cmp('data\ecoli\Fake_Ecoli.txt', 'data\ecoli\Fake_Ecoli_sbe_decompressed.txt')

11671499
4638690


True

## Let's split the file altogether, so it could take less space

By splitting A1001T10C1 into A4T2C and 100110, it's much easier to save space by turning the whole sequence into 2 separate file. We can discard number in last letter altogether!

In [1]:
from sbe import SBE_f,SBE_b,SBE_convert_f,SBE_convert_b,SBE_f_2file,SBE_b_2file

temp = SBE_convert_f('ATCATASFEGATCG')
SBE_convert_b(temp)

temp1,temp2 = SBE_f_2file([15,16,15,14,14,14,15,16,15,14,14,14])
print(temp1)
print(temp2)
SBE_b_2file(temp1,temp2)

[15, 12, 16, 8, 14, 6]
[1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]


array([15., 16., 15., 14., 14., 14., 15., 16., 15., 14., 14., 14.])

In [2]:
import arithc as arith
import fqt, ppm
import contextlib, sys
import filecmp
from IPython.display import clear_output

In [3]:
def SBEcompress(inp, bitout):
    initfreqs = fqt.FlatFrequencyTable(257)
    model = fqt.SimpleFrequencyTable(initfreqs)
    enc = arith.ArithmeticCoder(32)
    enc.start_encode(bitout) # New line!
    s = []
    while True:
        symbol = inp.read(1)
        if len(symbol) == 0:
                break
        s += [symbol[0]+2] ## leave room for 0,1!, Dont worry, we only compress 0,1
    sbe1,sbe2 = SBE_f_2file(s)
    print(len(sbe2))
    idx1 = 1
    idx2 = 0
    curr_size = sbe1[idx1]
    # prior (25,75) then (25,50) then (25,25)
#    k = len(sbe1)//2 # for DNA, k = 4
#     model.set(0, (k-1)*sbe1[idx1]//k)
#     model.set(1, sbe1[idx1]//k)

    model.set(1, sbe1[idx1]-sbe1[idx1+2])
    model.set(0, sbe1[idx1+2])
    print(model.get(1))
    print(model.get(0))
    while True:
        # Read and encode one byte
        if idx2 == len(sbe2):
            break
        symbol = sbe2[idx2]
        #print(symbol)
        idx2 += 1
        if idx2 % (len(sbe2)//10) == 0:
            print(str(10*idx2//(len(sbe2)//10)) + ' percent done')
            clear_output(wait = True)
        
        if idx2 > curr_size:
            #k -= 1
            idx1 += 2
            model.set(1, sbe1[idx1]-sbe1[idx1+2])
            model.set(0, sbe1[idx1+2])
            print(model.get(1))
            print(model.get(0))
            
            curr_size += sbe1[idx1]
            
        t = model.get_total() ## New lines!
        l = model.get_low(symbol)
        h = model.get_high(symbol)
        enc.storeRegion(l,h,t) 
        model.set(symbol, model.get(symbol)-1)
#         model.frequencies[symbol] -= 1
#         model.total -= 1
        #model.increment(symbol)
    t = model.get_total() ## New lines!
    l = model.get_low(256)
    h = model.get_high(256)
    enc.storeRegion(l,h,t)
    enc.finish_encode(bitout)  # New line!
    
    return sbe1

inputfile, outputfile = 'data\ecoli\Fake_Ecoli.txt', 'data\ecoli\Fake_Ecoli_sbe2.txt'
companionfile = 'data\ecoli\companion_Fake_Ecoli_sbe2.txt'
#Perform file compression
with open(inputfile, "rb") as inp, \
        contextlib.closing(arith.BitOutputStream(open(outputfile, "wb"))) as bitout:
    sbe1 = SBEcompress(inp, bitout)
    with open(companionfile, "w") as text_file:
        text_file.write(str(sbe1))
    

100 percent done


## This is the same as if compressed using the table, of course it is better than bzip2 and gzip but not as good as gtz